#### Задача 1. 

Напишите метакласс с логированием: при определении любого нового класса в файл (путь к которому хранится в статическом атрибуте метакласса) записывается имя этого класса, его суперклассы и атрибуты. 

In [38]:
class LogMeta(type):
    log_file = 'log.txt'

    def __new__(cls, name, bases, attrs):

        with open(cls.log_file, 'a') as file:

            file.write(f'Имя класса: {name}\n\n')
            file.write(f'Супер-пупер классы: {", ".join([base.__name__ for base in bases])}\n\n')
            file.write('Атрибуты: \n')

            for attr, value in attrs.items():
                file.write(f'{attr}: {value}\n')
                
            file.write('\n')

        return super().__new__(cls, name, bases, attrs)

In [39]:
class VeryNiceClass (metaclass=LogMeta):
    hi = 'Hello!!!'
    num = 42

    def __init__(self):
        self.not_hi = 'Byeee!'

    def method(self):
        print('Hello, world!')
        print(42, 42, 42)

In [42]:
with open(LogMeta.log_file, 'r') as file:
    print(file.read())

Имя класса: VeryNiceClass

Супер-пупер классы: 

Атрибуты: 
__module__: __main__
__qualname__: VeryNiceClass
hi: Hello!!!
num: 42
__init__: <function VeryNiceClass.__init__ at 0x0000022E23A4D6C0>
method: <function VeryNiceClass.method at 0x0000022E23556480>




#### Задача 2. 

Напишите метакласс, который любому новому классу будет добавлять метод sayHello(\*args, \*\*kwargs). 

In [53]:
class sayHelloMeta(type):

    def __new__(cls, name, bases, attrs):

        attrs['sayHello'] = cls.say_hello

        return super().__new__(cls, name, bases, attrs)

    @staticmethod
    def say_hello(*args, **kwargs):

        print('say Helloooooo!!!')

In [54]:
class VeryNiceClass(metaclass=sayHelloMeta):
    hi = 'Hello!!!'
    num = 42

    def __init__(self):
        self.not_hi = 'Byeee!'

    def method(self):
        print('Hello, world!')
        print(42, 42, 42)

In [55]:
hehe = VeryNiceClass()
hehe.sayHello()  

say Helloooooo!!!


#### Задача 3. 

Напишите метакласс, который позволяет своим классам иметь только атрибуты name, surname, age и никаких других. 

In [81]:
class NoAllowedAttrsMeta(type):

    def __new__(cls, name, bases, attrs):

        allowed_attrs = {'name', 'surname', 'age', '__module__', '__qualname__', '__init__'} #без этих штук не работало
        
        for attr in attrs:
            if attr not in allowed_attrs:
                raise AttributeError(f'Класс "{name}" нельзя добавить атрибут "{attr}", могут быть только атрибуты: name, surname, age')
        
        return super().__new__(cls, name, bases, attrs)


In [82]:
class Person(metaclass=NoAllowedAttrsMeta):
    name = 'Vasya'
    surname = 'Lingvistov'
    age = 22
    
    def __init__(self, name, surname, age):
        self.name = name
        self.surname = surname
        self.age = age

In [83]:
Person.name

'Vasya'

In [86]:
class Person_2(metaclass=NoAllowedAttrsMeta):
    name = 'Vasya'
    surname = 'Lingvistov'
    age = 22
    job = 'Linguist'
    
    def __init__(self, name, surname, age, job):
        self.name = name
        self.surname = surname
        self.age = age 
        self.job = job

AttributeError: Класс "Person_2" нельзя добавить атрибут "job", могут быть только атрибуты: name, surname, age

#### Задача 4 (2 балла). 

Приготовимся в большом проекте создать игру-стратегию (графику потом дизайнеры добавят). Наша игра будет называться Dawn of Man, и задачей игрока будет управлять поселением первобытных людей, которые в процессе набирают очки знаний и открывают новые технологии. Наши люди - это отдельные юниты, которые умеют взаимодействовать с элементами окружающей среды: у них будут какие-то инструменты для создания еды, оружия и прочего (например, какой-нибудь навес, под которым человек садится, набрав палок и камней, и ваяет себе палку-копалку). 

Наша задача - набросать черновик для игры: создать абстрактные классы для людей, объектов, с которыми они взаимодействуют, чтобы создавать вещи, и собственно вещей. Возможно, имеет смысл также предусмотреть черновик-класс для животных, на которых люди будут охотиться. 

Попробуйте также создать конкретный класс для кого-нибудь из них. 

In [124]:
from abc import ABC, abstractmethod

class Unit(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def interaction_with_environment(self, environment):
        pass

class Object(ABC):
    @abstractmethod
    def create_item(self, item):
        pass

class Item(ABC):
    @abstractmethod
    def use(self, unit):
        pass

class Human(Unit):
    def interaction_with_environment(self, environment):
        print(f'Человечек {self.name} взаимодействует с {environment}')

class Tool(Item):
    def use(self, unit):
        print(f'Человечек {unit.name} использует инструменты')

class Food(Item):
    def use(self, unit):
        print(f'Человечек {unit.name} вкусненько кушоет')

class Animal(ABC):
    @abstractmethod
    def hunt(self, unit):
        pass

#конкретный класс камень
class Stone(Object):
    def create_item(self, item):
        print(f'{item} создается с помощью камня')

#конкретный класс мамонт
class Mammoth(Animal):
    def hunt(self, unit):
        print(f'Человечек {unit.name} охотится на мамонта')

In [127]:
def Dawn_of_Man():
    human = Human('Васёк')
    tool = Tool()
    tool.use(human)
    stone = Stone()
    stone.create_item('палка-копалка')
    palka_kopalka = Tool()
    palka_kopalka.use(human)
    mammoth = Mammoth()
    mammoth.hunt(human)
    food = Food()
    food.use(human)

Dawn_of_Man()

Человечек Васёк использует инструменты
палка-копалка создается с помощью камня
Человечек Васёк использует инструменты
Человечек Васёк охотится на мамонта
Человечек Васёк вкусненько кушоет
